In [ ]:
import belasting as blst
import pandas as pd    

### Employee (non-ZZP)

In [ ]:
year = 2025

# annual (from Jaaropgave)
jaar_data = {
    'Applyer_1': {
        'company': 'Example_1 BV ',
        'annual_income': 1000,
        'paid_tax': 100},
    'Applyer_2': {
        'company': "Example_2 BV",
        'annual_income': [600, 700],
        'paid_tax': [25, 47]},
}

apply_zvw = False  # Health Insurance is not applied (paid by employer)

In [ ]:

res = []

for name, year_data in jaar_data.items():

    if isinstance(year_data['annual_income'], list):
        year_data['annual_income'] = sum(year_data['annual_income'])
    if isinstance(year_data['paid_tax'], list):
        year_data['paid_tax'] = sum(year_data['paid_tax'])

    tax_data = blst.box1_tax_calculate(
        income=year_data['annual_income'],
        year=year, 
        apply_zvw=apply_zvw
        )
    year_data['Name'] = name
    year_data.update(tax_data)
    res.append(year_data)

    
df_ = pd.DataFrame(res)
df_ = df_.drop(df_.columns[(df_ == 0).all(axis=0)], axis=1).set_index('Name')  # Remove zero-ed columns
df_ = df_[['year'] + [c for c in df_.columns if c != 'year']]    # move `year` column to the first position
df_

#### Tax return

In [ ]:
tr = df_['paid_tax'] - df_['box1_tax_netto']
tr.name = 'Return'
df_tr = tr.to_frame()

df_tr.style.map(
    lambda val: f'color: {"green" if val > 0 else "red" if val < 0 else "black"}; font-weight: bold'
).format("{:.2f}")


### ZZP

In [ ]:
year = 2026

revs = range(40000, 110000, 5000)  # income revenues

zzp = True
apply_zvw = False      # reimbursement
costs = {
    'business': 1500,
    'pension': 3000
    }

In [ ]:
res = [blst.box1_tax_calculate(
            rev, year, True, costs, apply_zvw=apply_zvw) for rev in revs]
df = pd.DataFrame(res)
df = df.drop(df.columns[(df == 0).all(axis=0)], axis=1).set_index('annual_income')  # Remove zero-ed columns

df

In [ ]:
df.plot(
    kind='line',
    # x='annual_income',
    y='income_netto_monthly',
    title=f'Income netto monthly ({year})' + '_ZZP' * zzp,
    xlabel='Annual income',
    ylabel='Income netto monthly',
    figsize=(10, 6),
    grid=True,
    legend=False
    );